In [ ]:
import torch
import botorch
import numpy as np
import matplotlib.pyplot as plt
import utilities as util

from transformed_model import KeyedTransformedModel

In [ ]:
# load surrogate and define objective
surrogate = util.Surrogate()
Objective = util.NegativeTransverseBeamSize
ground_truth = Objective(surrogate.model)

In [ ]:
# load correlated models
n_epochs = 25
corr_models, corr_objs = [], []
for i in range(n_epochs):
    corr_models.append(util.load_correlated_model(i + 1, surrogate)[0])
    corr_objs.append(Objective(corr_models[-1]))

# Correlations and MAEs

In [ ]:
path = "./corr_models/"
rng_seed = 3
n_samples = 10000
cutoff_value = None

# uniform samples
x = surrogate.sample_x(n_samples, seed=rng_seed)
y = ground_truth(x).detach()

# calculate correlations and MAE for all models
correlations, maes = [], []
for i in range(n_epochs):
    y_corr = corr_objs[i](x).detach()
    correlations.append(util.calc_correlation(y, y_corr, cutoff_value))
    maes.append(util.calc_mean_absolute_error(y, y_corr))
torch.save(torch.tensor(correlations), path + "correlations.pt")
torch.save(torch.tensor(maes), path + "maes.pt")

In [ ]:
# find best models
idx_best_corr = np.argmax(correlations)
idx_best_mae = np.argmin(maes)

# plot correlations and MAEs
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
ax[0].plot(torch.arange(n_epochs) + 1, correlations, "C0o:")
ax[0].plot(idx_best_corr + 1, correlations[idx_best_corr], "C3o", 
           label="{:.2f} (after {:d} epochs)".format(correlations[idx_best_corr], idx_best_corr + 1))
ax[0].set_xlabel("Number of Epochs")
ax[0].set_ylabel("Correlation")
ax[0].legend()

ax[1].plot(torch.arange(n_epochs) + 1, torch.tensor(maes) * ground_truth.unit_factor, "C0o:")
ax[1].plot(idx_best_mae + 1, maes[idx_best_mae] * ground_truth.unit_factor, "C3o", 
           label="{:.2f}$\,${} (after {:d} epochs)".format(maes[idx_best_mae] * ground_truth.unit_factor, 
                                                           ground_truth.unit[1:-1], idx_best_mae + 1))
ax[1].set_xlabel("Number of Epochs")
ax[1].set_ylabel("MAE {}".format(ground_truth.unit))
ax[1].legend()

fig.tight_layout()

# Analysis of Best Model

In [ ]:
# select model
idx_best = idx_best_corr
corr_obj = corr_objs[idx_best]
y_corr = corr_obj(x).detach()

# print correlation
print("{:d} Epochs -> Correlation: {:.2f}, MAE: {:.2f} {}".format(idx_best + 1, correlations[idx_best], 
                                                                  maes[idx_best] * ground_truth.unit_factor,
                                                                  ground_truth.unit[1:-1]))

In [ ]:
# plot objective distributions
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(10, 5))
ax[0].hist(y * ground_truth.unit_factor, bins=100)
ax[0].set_title("Surrogate Model")
ax[0].set_ylabel("Number of Samples")
ax[1].hist(y_corr * ground_truth.unit_factor, bins=100)
ax[1].set_title("Correlated Model ({:d} Epochs)".format(idx_best + 1))
for i in range(2):
    ax[i].set_xlabel("{} {}".format(ground_truth.name, ground_truth.unit))
fig.tight_layout()

In [ ]:
# plot correlations
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
ax.set_xlabel("Correlated Model Objective {}".format(ground_truth.unit))
ax.set_ylabel("Surrogate Model Objective {}".format(ground_truth.unit))
ax.set_title("Correlation ({:d} Epochs)".format(idx_best + 1))
if cutoff_value is not None:
    ax.set_xlim([cutoff_value * ground_truth.unit_factor, 0.0])
    ax.set_ylim([cutoff_value * ground_truth.unit_factor, 0.0])
ax.scatter(y_corr * ground_truth.unit_factor, y * ground_truth.unit_factor, s=np.full(n_samples, 10))
ax.scatter(y * ground_truth.unit_factor, y * ground_truth.unit_factor, s=np.full(n_samples, 10), alpha=0.5)
fig.tight_layout()

In [ ]:
# use sample optimum as reference point
ref_idx = torch.argmax(y)
ref_x, ref_y = x[ref_idx], y[ref_idx]

# 1D scan reference point
def scan_ref_x(n_samples, dim=0):
    x_lim = surrogate.raw_x_lim
    samples = ref_x.repeat(n_samples, 1)
    samples[:, dim] = torch.linspace(x_lim[dim, 0], x_lim[dim, 1], n_samples)
    return samples.double()

In [ ]:
# correlation for individual input dimensions
n_scan = 100
nrows, ncols = 5, 3
figsize = (12, 12 * nrows / ncols)

fig, ax = plt.subplots(nrows=nrows, ncols=ncols, sharey=True, figsize=figsize)
idx = [i for i in range(surrogate.x_dim) if i not in surrogate.fixed_feature_columns]  # remove fixed dimensions

for i in range(nrows * ncols):
    ax_i = ax[i // ncols, i % ncols]
    if i > len(idx) - 1:
        ax_i.axis('off')
    else:
        j = idx[i]
        # calculate 1D scan
        x_scan = scan_ref_x(n_scan, dim=j)
        y_scan = ground_truth(x_scan).detach()
        y_corr_scan = corr_obj(x_scan).detach()
        corr = util.calc_correlation(y_scan, y_corr_scan, cutoff_value)
        mae = util.calc_mean_absolute_error(y_scan, y_corr_scan, cutoff_value)

        # map sim values to pv
        parameter_name = surrogate.model_info["model_in_list"][j]
        pv_name = surrogate.pv_info["sim_name_to_pv_name"][parameter_name]
        pv_factor = surrogate.pv_info["sim_to_pv_factor"][parameter_name]
        pv_unit = surrogate.pv_info["pv_unit"][pv_name]

        # plot data
        ax_i.plot(x_scan[:, j] * pv_factor, y_scan * ground_truth.unit_factor, "C0", label="Surrogate")
        ax_i.plot(x_scan[:, j] * pv_factor, y_corr_scan * ground_truth.unit_factor, "C1", 
                  label="Correlation: {:.0f}$\,$%\nMAE: {:.2f}$\,${}".format(100 * corr, 
                                                                             mae * ground_truth.unit_factor,
                                                                             ground_truth.unit[1:-1]))
        ax_i.set_xlabel("{} {}".format(pv_name, pv_unit))
        if i % 3 == 0:
            ax_i.set_ylabel("{} {}".format(ground_truth.name, ground_truth.unit))
        ax_i.legend()
fig.tight_layout()